In [ ]:
!pip install feedparser requests beautifulsoup4 langchain langchain-groq langchain-community faiss-cpu sentence-transformers
!pip install python-dotenv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=e1ff938dbef639caa2d443edec80aa7e0dafb77aec13ddef91eb4dd453f280bc
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
     

In [ ]:
import os
import requests
import feedparser
from urllib.parse import quote
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
def get_content(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.find_all("p")
    content = " ".join(p.get_text() for p in paragraphs)
    return content.strip()

In [ ]:
def get_news(query, num=10):
    rss_url = f"https://news.google.com/rss/search?q={quote(query)}"
    feed = feedparser.parse(rss_url)
    articles = []
    for item in feed.entries[:num]:
        content = get_content(item.link)
        if not content:
            content = item.get("summary", "")
        articles.append({
            "title": item.title,
            "link": item.link,
            "published": item.get("published", ""),
            "content": content
        })
    return articles

In [ ]:
queries = [
    "Apple stock",
    "Microsoft stock",
    "Google stock",
    "Amazon stock",
    "Tesla stock"
]
documents = []
for q in queries:
    news = get_news(q, num=10)
    for article in news:
        if article["content"].strip():
            documents.append(
                f"Title: {article['title']}\n"
                f"Date: {article['published']}\n"
                f"Source: {article['link']}\n"
                f"{article['content']}"
            )

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vector_db = FAISS.from_texts(documents, embeddings)

In [ ]:
os.environ["GROQ_API_KEY"] = "API_KEY"
llm = ChatGroq(
    model="llama-3.1-8b-instant"
)

In [ ]:
def get_context(query, k=3):
    docs = vector_db.similarity_search(query, k=k)
    return "\n\n".join(d.page_content for d in docs)
def get_intent(query):
    q = query.lower()
    if "why" in q:
        return "explanation"
    if "when" in q:
        return "trend"
    if "up" in q or "down" in q or "rise" in q or "fall" in q:
        return "movement"
    return "general"
def get_ticker(query):
    q = query.lower()
    if "apple" in q:
        return "AAPL"
    if "microsoft" in q:
        return "MSFT"
    if "google" in q:
        return "GOOGL"
    if "amazon" in q:
        return "AMZN"
    if "tesla" in q:
        return "TSLA"
    return "UNKNOWN"

In [ ]:
prompt = PromptTemplate(
    input_variables=["ticker", "intent", "context", "question"],
    template="""
You are a financial analyst.

Stock Ticker: {ticker}
User Intent: {intent}

Relevant News Context:
{context}

Question:
{question}

Answer clearly and factually using ONLY the context above.
"""
)

In [ ]:
def agent(query):
    intent = get_intent(query)
    ticker = get_ticker(query)
    context = get_context(query)
    final_prompt = prompt.format(
        ticker=ticker,
        intent=intent,
        context=context,
        question=query
    )
    response = llm.invoke(final_prompt)
    return response.content

In [ ]:
query = "Why did Apple stock go down recently?"
answer = agent(query)
print("ANSWER:", answer)

ANSWER: Based on the provided news context, it appears that Apple stock has been experiencing a 7-week losing streak. However, the exact reason for this decline is not explicitly stated in the news articles.

The articles from TipRanks, CNBC, and Yahoo Finance focus on the potential for Apple stock to recover and potentially reach higher prices, such as $300 or $350, despite various challenges like tariffs, regulation, and macro uncertainty. They do not provide a clear explanation for the recent decline.

As a financial analyst, I would recommend further research to determine the specific factors contributing to Apple's recent stock performance. This could involve analyzing financial statements, industry trends, and market news to identify potential causes for the decline.
